# Basics: Node and MessageNode

`trace` is a comptuational grpah framework for tracing and optimizing codes. Its core data structure is the "node" container of python objects. To create a node, use `node` method, which creates a `Node` object. To access, the content of a node, use the `data` attribute.

In [6]:
from autogen.trace import node

x = node(1)  # node of int
print("node of int", x.data)
x = node("string")  # node of str
print(x.data)
x = node([1, 2, 3])  # node of list
print(x.data)
x = node({"a": 1, "b": 2})  # node of dict
print(x.data)


class Foo:
    def __init__(self, x):
        self.x = x
        self.secret = "secret"

    def print(self, val):
        print(val)


x = node(Foo("foo"))  # node of a class instance
print(x.data)

node of int 1
string
[1, 2, 3]
{'a': 1, 'b': 2}


When a computation is performed using the contents of nodes, the result is also a node. This allows for the creation of a computation graph. The computation graph is a directed acyclic graph where the edges indicate the data dependencies.

Nodes that are defined manually can be marked as trainable by setting their `trainable` attribute to True; such nodes are a subclass of Node called `ParameterNode`.
Nodes that are created automatically as a result of computations are a different subclass of Node called `MessageNode`.

Nodes can be copied. This can be done in two ways with `clone` or `detach`

In [7]:
# clone returns a MessageNode whose parent is the original node
x_clone = x.clone()
assert x in x_clone.parents
assert x_clone.data != x.data
assert x_clone.data.x == x.data.x
print(x_clone.data)
# detach returns a new Node which is not connected to the original node
x_detach = x.detach()
assert len(x_detach.parents) == 0
assert x_detach.data != x.data
assert x_detach.data.x == x.data.x

`trace` overloads python's magic methods that gives return value explicitly (such as `__add__`), except logical operations such as `__bool__` and setters. (The comparison magic methods compares the level of the nodes in the global graph, rather than comparing the data.) 

When nodes are used with these magic methods, the output would be a `MessageNode`, which is a subclass of `Node` that has the inputs of the method as the parents. The attribute `description` of a `MessageNode` documents the method's function.

In [8]:
def print_node(node):
    print(node)
    print(f"parents: {[p.name for p in node.parents]}")


# Basic arithmetic operations
x = node(1, name="node_x")
y = node(3, name="node_y")
z = x / y
z2 = x / 3  # the int 3 would be converted to a node automatically
print(z)
print_node(z)
print("\n")

# Index a node
dict_node = node({"a": 1, "b": 2}, name="dict_node")
a = dict_node["a"]
print_node(a)
print("len(dict_node) =", dict_node.len())

print("\n")

# Getting class attribute and calling class method
x = node(Foo("foo"))
x.call("print", "hello world")
print_node(x.getattr("secret"))

MessageNode: (divide:2, dtype=<class 'float'>, data=0.3333333333333333)
MessageNode: (divide:2, dtype=<class 'float'>, data=0.3333333333333333)
parents: ['node_x:1', 'node_y:1']


MessageNode: (getitem:1, dtype=<class 'int'>, data=1)
parents: ['dict_node:1', 'str:12']
len(dict_node) = MessageNode: (len:1, dtype=<class 'int'>, data=2)


Node: (str:14, dtype=<class 'str'>, data=hello world)
MessageNode: (getattr:9, dtype=<class 'str'>, data=secret)
parents: ['Foo:4', 'str:15']


Nodes can not be used in logical operations like and, or, not. This is an explicit design choice so as to ensure that logical operations in python code is explicitly traced.

In [9]:
x = node(True)
try:
    if x:
        print("True")
except Exception as e:
    print(e)
    print("Use if x.data instead of if x")


x = node([1, 2, 3])
try:
    1 in x
except Exception as e:
    print(e)
    print("Use 1 in x.data instead of 1 in x")

True


Nodes can be used to encapsulate any python object, including functions. Here're a few examples

In [10]:
def fun(x):
    return x + 1


fun_node = node(fun)
y = fun_node(node(1))
print(f"output: {y}\nparents {[(p.name, p.data) for p in y.parents]}")
print("\n\n")


class Foo:

    def __init__(self):
        self.node = node(1)
        self.non_node = 2

    def trace_fun(self):
        return self.node * 2

    def non_trace_fun(self):
        return self.non_node * 2


foo = node(Foo())

try:
    foo.node
    foo.trace_fun()
except AttributeError:
    print("The attribute of the wrapped object cannot be directly accessed. Instead use getattr() or call()")


attr = foo.getattr("node")
print(f"foo_node: {attr}\nparents {[(p.name, p.data) for p in attr.parents]}")


attr = foo.getattr("non_node")
print(f"non_node: {attr}\nparents {[(p.name, p.data) for p in attr.parents]}")


fun = foo.getattr("non_trace_fun")
y = fun()
print(f"output: {y}\nparents {[(p.name, p.data) for p in y.parents]}")

try:
    fun = foo.getattr("trace_fun")
    y = fun()
except AssertionError as e:
    print(e)

y = foo.call("non_trace_fun")
print(f"output: {y}\nparents {[(p.name, p.data) for p in y.parents]}")

try:
    y = foo.call("trace_fun")
except AssertionError as e:
    print(e)

output: MessageNode: (call:7, dtype=<class 'int'>, data=2)
parents [('function:1', <function fun at 0x7f39cef99310>), ('int:11', 1)]



The attribute of the wrapped object cannot be directly accessed. Instead use getattr() or call()
foo_node: MessageNode: (getattr:10, dtype=<class 'int'>, data=1)
parents [('Foo:5', <__main__.Foo object at 0x7f38fe04da90>), ('str:16', 'node')]
non_node: MessageNode: (getattr:11, dtype=<class 'int'>, data=2)
parents [('Foo:5', <__main__.Foo object at 0x7f38fe04da90>), ('str:17', 'non_node')]
output: MessageNode: (call:8, dtype=<class 'int'>, data=4)
parents [('getattr:12', <bound method Foo.non_trace_fun of <__main__.Foo object at 0x7f38fe04da90>>)]
output: MessageNode: (call:10, dtype=<class 'int'>, data=4)
parents [('getattr:14', <bound method Foo.non_trace_fun of <__main__.Foo object at 0x7f38fe04da90>>)]


TraceExecutionError: TraceExecutionError: (AssertionError) The function must be callable.

## Writing Custom Node Operators
In addition to magical methods, we can use `trace_op` to write custom methods that are traceable. When decorating a method with `trace_op`, it needs a description of the method. It has a format of `[method_name] description`. `trace_op` will automatically add all nodes whose `data` attribute is used within the function as the parents of the output `MessageNode`.

Given a function `fun`, the decorated function `trace_op(description)(fun)` by default will unpack all the inputs (it unpacks all node containers), send them to `fun`, and then creates a `MessageNode` to wrap the output of `fun` which has parents containing all the nodes used in this operation. 

Since all inputs are unpacked, they will be set as the parents. The user can override this behavior by setting `trace_op(description, unpack_input=False)`, which would let `fun` see the original inputs.


In [ ]:
from autogen.trace import trace_op


@trace_op("[add_1] Add 1 to input x")
def foo(x):
    return x + 1


x = node(1, name="node_x")
z = foo(x)
print_node(z)
print("\n")


@trace_op("[add] Add input x and input y")
def foo(x, y):
    return x + y


x = node(1, name="node_x")
y = node(2, name="node_y")
z = foo(x, y)
print_node(z)
print("\n")

# The output is a node of a tuple of two nodes


@trace_op("[pass_through] No operation, just return inputs")
def foo(x, y):
    return x, y


x = node(1, name="node_x")
y = node(2, name="node_y")
z = foo(x, y)
print(z)
from autogen.trace.nodes import Node

assert isinstance(z, Node)
assert isinstance(z.data, tuple)
assert len(z.data) == 2
print("\n")


# The output is a tuple of two nodes
@trace_op("[pass_through] No operation, just return inputs", n_outputs=2)
def foo(x, y):
    return x, y


x = node(1, name="node_x")
y = node(2, name="node_y")
z = foo(x, y)
print(z)
assert isinstance(z, tuple)
assert len(z) == 2

### Describing Relationship between Inputs and Outputs and Nodes in the Graph
One can additionally provide `node_dict` to specify how each variable mentioned in `description` is related to the nodes in the graph. This relationship is stored in the `inputs` attribute of `MessageNode`. See examples 
below.

In [ ]:
# The default value of node_dict is None. In this case, the key of the inputs dict is the name of the input nodes.
@trace_op("[add_1] Add 1 to input x")
def foo(x):
    return x + 1


z = foo(x)
print({k: (v.name, v.data) for k, v in z.inputs.items()})

# When node_dict is set to 'auto', the key of the inputs dict is the name specified in the function signature.


@trace_op("[add_1] Add 1 to input x", node_dict="auto")
def foo(input):
    return input + 1


z = foo(x)
print({k: (v.name, v.data) for k, v in z.inputs.items()})

# When node_dict is set to a dict, the key of the inputs dict is the name specified in the dict.
node_dict = {"custom_x": x}


@trace_op("[add_1] Add 1 to input x", node_dict=node_dict)
def foo(x):
    return x + 1


z = foo(x)
print({k: (v.name, v.data) for k, v in z.inputs.items()})

Using node_dict is useful when the function uses nodes that are not in the function signature.


In [ ]:
# By default, the inputs dict only contains the nodes that are in the function signature. One can update the inputs dict by using node_dict.
x = node(1, name="node_x")
y = node(2, name="node_y")


@trace_op("[add_1] Add input x to node_y.", node_dict="auto")
def foo(x):
    return x + y.data


try:
    z = foo(x)
except Exception as e:
    # Since the function signature does not contain y, the function will raise an error.
    print(e)
# We can use node_dict to add y to the inputs dict.
node_dict = {"node_y": y}


@trace_op("[add_1] Add input x to node_y.", node_dict=node_dict)
def foo(x):
    return x + y.data


z = foo(x)
print({k: (v.name, v.data) for k, v in z.inputs.items()})

# Visualize Graph

The graph of nodes can be visualized by calling `backward` method of a node. (Later we will cover how `backward` also sends feedback across the graph). 

In [ ]:
from autogen.trace.nodes import GRAPH

GRAPH.clear()  # to remove all the nodes
x = node(1, name="node_x")
y = node(2, name="node_y")
a = x + y
b = x + 1
final = a + b
final.backward(visualize=True)

In [ ]:
GRAPH.clear()
x = node(True)
one = node(1)
zero = node(0)
print(x, one, zero)
# Logical operations are not traceable
y = one if x.data else zero
y.backward(visualize=True)

In [ ]:
# This is traceable
node_dict = {"one": one, "zero": zero}


@trace_op("[fun] Return one if input x is True, otherwise return zero", node_dict=node_dict)
def fun(x):
    return one.data if x else zero.data


y = fun(x)
y.backward(visualize=True)

# Broadcasting
Using `apply_op`, we can broadcast node operators to a container of nodes. A container of nodes are either `list`, `tuple`, `dict`, or subclass of an abstract class `BaseModule`. `apply_op` recursively applies the operator to all nodes in the container. 

In [ ]:
from autogen.trace import apply_op, node, NodeContainer
from autogen.trace import operators as ops

import copy

# Using list as a node container
x = [node(1), node(2), 1]
y = [node(3), node(4), 2]
z = copy.deepcopy(x)
z = apply_op(ops.add, z, x, y)
print("x", [x[0].data, x[1].data, x[2]])
print("y", [y[0].data, y[1].data, y[2]])
print("Elements in z should be added, except for the last one. Value: ", [z[0].data, z[1].data, z[2]])


# Using list as a node container
x = dict(a=node(1), b=0)
y = dict(a=node(3), b=0)
z = copy.deepcopy(x)
z = apply_op(ops.add, z, x, y)
print(f"{x['a'].data}+{y['a'].data}={z['a'].data}")
print(f"{x['b']}=={y['b']}=={z['b']}")

# Using a custom class as a node container


class Foo(NodeContainer):
    def __init__(self, x):
        self.x = node(x)
        self.y = [node(1), node(2)]
        self.z = 1


x = Foo("x")
y = Foo("y")
x_plus_y = Foo("template")
x_plus_y = apply_op(ops.add, x_plus_y, x, y)
print("x_plus_y.x should be added. Value: ", x_plus_y.x.data)
print("x_plus_y.y should be added. Value: ", [n.data for n in x_plus_y.y])
print("x_plus_y.z should be not added, just 1. Value: ", x_plus_y.z)

## Nodes and Python Data Structure

We can create a `node` over Python data structure like dictionary, tuple, set, or list. We automatically handle the iteration and you can wrap a node around any data structure and use them like normal python objects.

In [ ]:
from autogen.trace import node

args = node({"arg1", "arg2"}, trainable=False)
for a in args:
    print(a)

a.backward(visualize=True)

In [ ]:
parms = node([1, 2], trainable=False)
args = node(["arg1", "arg2"], trainable=False)

for a, p in zip(args, parms):
    print(a, p)

p.backward(visualize=True)